<a href="https://colab.research.google.com/github/neopian-sys/Week3answersneopiansys/blob/main/w05_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 LSTM

Title: Character-level recurrent sequence-to-sequence model<br/>
Author: [fchollet](https://twitter.com/fchollet)<br/>
Date created: 2017/09/29<br/>
Last modified: 2023/11/22<br/>
Description: Character-level recurrent sequence-to-sequence model.<br/>
Accelerator: GPU<br/>

References:
* https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
* https://github.com/keras-team/keras-io/blob/master/examples/nlp/lstm_seq2seq.py


## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

![](seq2seq-inference.png "seq2seq")

In [1]:
# Setup
import numpy as np
import keras
from pathlib import Path
import re
import random
import tensorflow as tf

# Get the data

Feel free to use a different language from https://www.manythings.org/anki/

In [2]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2025-02-19 14:57:52--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  6.43MB/s    in 1.2s    

2025-02-19 14:57:54 (6.43 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]



In [3]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [4]:
!ls

_about.txt  fra-eng.zip  fra.txt  sample_data


## Prepare the data


In [5]:
# Get the data
# Sourced from https://www.manythings.org/anki/
fpath = "./fra.txt"
data_path = Path(fpath).absolute()

# Set some key parameters
batch_size = 64  # Batch size for training.

#TODO: Start with a small number of epochs (maybe 5) if running locally on CPU
epochs = 5  # Number of epochs to train for.

latent_dim = 512  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Bonus: What happens with longer texts? Why?
MAX_INPUT_LENGTH=15


In [6]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set() # Unique characters in input_texts
target_characters = set() # Unique characters in target_texts

RE_W = re.compile('[\W_]+', re.UNICODE)

# We use "tab" (\t) as the "start sequence" character
# for the targets, and newline ("\n") as "end sequence" character.

START_SEQ = "\t"
END_SEQ = "\n"

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

random.seed(42)
random.shuffle(lines) # Put the lines in a different order

seen_inputs = set()
for line in lines:
    try:
        input_text, target_text, _ = line.split("\t")
    except:
        continue

    # Skip longer texts -- can you guess why?
    if len(input_text) > MAX_INPUT_LENGTH:
        continue

    # # Alternative, truncate long texts
    # if len(input_text) > MAX_INPUT_LENGTH:
    # input_text = input_text[:MAX_INPUT_LENGTH]
    # if len(target_text) > MAX_INPUT_LENGTH:
    #    target_text = target_text[:MAX_INPUT_LENGTH]


    # There appear to be a lot of duplicates in the data.
    # We'll ignore direct duplicates
    unq = RE_W.sub('', input_text)
    if unq in seen_inputs:
        continue
    seen_inputs.add(unq)

    # TODO: Prepend the START_SEQ and append the END_SEQ to the target_text
    # Prepend START_SEQ and append END_SEQ to the target text
    target_text = START_SEQ + target_text + END_SEQ

    input_texts.append(input_text)
    target_texts.append(target_text)

    #TODO: Update input_characters and target_characters with any new characters
    # Add new characters to input and target character sets
    input_characters.update(set(input_text))   # Add all unique input text characters
    target_characters.update(set(target_text)) # Add all unique target text characters

    # Finish if we have enough samples
    if len(input_texts) >= num_samples:
        break

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

# TODO: Make input_token_index and target_token_index dictionaries that map from characters to their indices
# Create dictionaries mapping characters to unique indices
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

Number of samples: 8017
Number of unique input tokens: 72
Number of unique output tokens: 94
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [7]:
list(zip(input_texts, target_texts))[:10]

[('Those are good.', '\tCeux-ci sont bons.\n'),
 ('I feel fine.', '\tJe me sens bien.\n'),
 ("I'm unemployed.", '\tJe suis au chômage.\n'),
 ('Hurry up.', '\tMagnez-vous !\n'),
 ('Give it a try.', '\tEssaie.\n'),
 ('Do it again!', '\tFaites-le de nouveau\u202f!\n'),
 ('He stood up.', '\tIl se mit debout.\n'),
 ('Tom was silent.', '\tTom gardait le silence.\n'),
 ("I'm finished.", "\tJ'ai terminé.\n"),
 ('Look at it.', '\tRegarde-le !\n')]

In [9]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


In [10]:
def get_nonzero_entries(matrix):
    return list(zip(*np.nonzero(matrix)))

In [11]:
print("Example input sentence:")
print(input_texts[0])

print("This is encoded as one-hot encodings; so, there are lots of zeros!")
print(encoder_input_data[0])

print("Here we print the coordinates of the non-zero values.")
print("There the first number is the position in the sentence")
print("The second is the mapping of a character to a number")
print(get_nonzero_entries(encoder_input_data[0]))

print("we see, for example, that u is encoded as a 1 in the 63rd position.")
print(input_token_index["u"])
print("this occurs in for the 5th character; so, we have (4, 61) in the output above")

Example input sentence:
Those are good.
This is encoded as one-hot encodings; so, there are lots of zeros!
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Here we print the coordinates of the non-zero values.
There the first number is the position in the sentence
The second is the mapping of a character to a number
[(0, 40), (1, 52), (2, 59), (3, 63), (4, 49), (5, 0), (6, 45), (7, 62), (8, 49), (9, 0), (10, 51), (11, 59), (12, 59), (13, 48), (14, 9)]
we see, for example, that u is encoded as a 1 in the 63rd position.
65
this occurs in for the 5th character; so, we have (4, 61) in the output above


In [ ]:
#TODO: Repeat the analysis above with `target_texts[0]` and
# analyse decoder_input_data[0] and decoder_output_data[0]
# Remember that each target text starts with a tab (\t) and ends with a newline (\n)

# Print the pairs of non-zero data for decoder_input_data[0] and decoder_target_data[0]
# on lines next to each other and understand the relationship between them.


In [12]:
print("Example target sentence:")
print(target_texts[0])  # The first French sentence in the dataset

Example target sentence:
	Ceux-ci sont bons.



In [13]:
print("\nOne-hot encoding for `decoder_input_data[0]` (target input):")
print(get_nonzero_entries(decoder_input_data[0]))


One-hot encoding for `decoder_input_data[0]` (target input):
[(0, 0), (1, 22), (2, 48), (3, 64), (4, 67), (5, 9), (6, 46), (7, 52), (8, 2), (9, 62), (10, 58), (11, 57), (12, 63), (13, 2), (14, 45), (15, 58), (16, 57), (17, 62), (18, 10), (19, 1), (20, 2), (21, 2), (22, 2), (23, 2), (24, 2), (25, 2), (26, 2), (27, 2), (28, 2), (29, 2), (30, 2), (31, 2), (32, 2), (33, 2), (34, 2), (35, 2), (36, 2), (37, 2), (38, 2), (39, 2), (40, 2), (41, 2), (42, 2), (43, 2), (44, 2), (45, 2), (46, 2), (47, 2), (48, 2), (49, 2), (50, 2), (51, 2), (52, 2), (53, 2), (54, 2), (55, 2), (56, 2), (57, 2), (58, 2)]


In [14]:
print("\nOne-hot encoding for `decoder_target_data[0]` (target output):")
print(get_nonzero_entries(decoder_target_data[0]))


One-hot encoding for `decoder_target_data[0]` (target output):
[(0, 22), (1, 48), (2, 64), (3, 67), (4, 9), (5, 46), (6, 52), (7, 2), (8, 62), (9, 58), (10, 57), (11, 63), (12, 2), (13, 45), (14, 58), (15, 57), (16, 62), (17, 10), (18, 1), (19, 2), (20, 2), (21, 2), (22, 2), (23, 2), (24, 2), (25, 2), (26, 2), (27, 2), (28, 2), (29, 2), (30, 2), (31, 2), (32, 2), (33, 2), (34, 2), (35, 2), (36, 2), (37, 2), (38, 2), (39, 2), (40, 2), (41, 2), (42, 2), (43, 2), (44, 2), (45, 2), (46, 2), (47, 2), (48, 2), (49, 2), (50, 2), (51, 2), (52, 2), (53, 2), (54, 2), (55, 2), (56, 2), (57, 2), (58, 2)]


In [15]:
print("\nComparing `decoder_input_data[0]` and `decoder_target_data[0]`:\n")
input_nonzero = get_nonzero_entries(decoder_input_data[0])
target_nonzero = get_nonzero_entries(decoder_target_data[0])

# Print input-output pairs
for inp, tgt in zip(input_nonzero, target_nonzero):
    print(f"Input: {inp} -> Target: {tgt}")


Comparing `decoder_input_data[0]` and `decoder_target_data[0]`:

Input: (0, 0) -> Target: (0, 22)
Input: (1, 22) -> Target: (1, 48)
Input: (2, 48) -> Target: (2, 64)
Input: (3, 64) -> Target: (3, 67)
Input: (4, 67) -> Target: (4, 9)
Input: (5, 9) -> Target: (5, 46)
Input: (6, 46) -> Target: (6, 52)
Input: (7, 52) -> Target: (7, 2)
Input: (8, 2) -> Target: (8, 62)
Input: (9, 62) -> Target: (9, 58)
Input: (10, 58) -> Target: (10, 57)
Input: (11, 57) -> Target: (11, 63)
Input: (12, 63) -> Target: (12, 2)
Input: (13, 2) -> Target: (13, 45)
Input: (14, 45) -> Target: (14, 58)
Input: (15, 58) -> Target: (15, 57)
Input: (16, 57) -> Target: (16, 62)
Input: (17, 62) -> Target: (17, 10)
Input: (18, 10) -> Target: (18, 1)
Input: (19, 1) -> Target: (19, 2)
Input: (20, 2) -> Target: (20, 2)
Input: (21, 2) -> Target: (21, 2)
Input: (22, 2) -> Target: (22, 2)
Input: (23, 2) -> Target: (23, 2)
Input: (24, 2) -> Target: (24, 2)
Input: (25, 2) -> Target: (25, 2)
Input: (26, 2) -> Target: (26, 2)
Input:

## Build the model


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense

# Define input layer for the encoder (one-hot character vectors)
encoder_inputs = Input(shape=(None, num_encoder_tokens))  # None allows variable sequence length

# Define LSTM encoder
encoder = LSTM(latent_dim, return_state=True)  # We only return the states
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the hidden and cell states
encoder_states = [state_h, state_c]  # Context vector

# Define input layer for the decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))  # One-hot character vectors for target text

# Define LSTM decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Define Dense layer with softmax activation
decoder_dense = Dense(num_decoder_tokens, activation="softmax")  # Predict next character
decoder_outputs = decoder_dense(decoder_outputs)

# Define the full training model
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

## Train the model


In [21]:
from keras.utils import register_keras_serializable

@register_keras_serializable(package="oii-sds-nlp")

def masked_accuracy(y_true, y_pred):
    # Convert y_true to float32 for safe division operations
    y_true = tf.cast(y_true, tf.float32)

    # ✅ Get the true class labels (argmax over last axis)
    y_true_class = tf.math.argmax(y_true, axis=-1)
    y_true_class = tf.cast(y_true_class, tf.float32)  # Ensure same type

    # ✅ Get the predicted class labels
    y_pred_class = tf.math.argmax(y_pred, axis=-1)
    y_pred_class = tf.cast(y_pred_class, tf.float32)  # Ensure same type

    # ✅ Create a mask (1 where at least one non-zero value exists)
    mask = tf.reduce_max(y_true, axis=-1)  # Find the max nonzero value per row
    mask = tf.cast(mask, tf.float32)  # Convert mask to float

    # ✅ Compare predictions with true labels
    matches_true_pred = tf.equal(y_true_class, y_pred_class)
    matches_true_pred = tf.cast(matches_true_pred, tf.float32)  # Convert to float

    # ✅ Apply the mask
    matches_true_pred *= mask

    # ✅ Compute masked accuracy: correct predictions / valid tokens
    masked_acc = tf.reduce_sum(matches_true_pred) / tf.reduce_sum(mask)

    return masked_acc


# Let's tests our function
y_true_test=np.array([[[1,0,0],[0,1,0],[1,0,0]]])
y_pred_test=np.array([[[1,0,0],[0,1,0],[0,0,1]]])

# Should have 100% if input and output match
t1 = masked_accuracy(y_true_test, y_true_test)
print("t1=", t1)
assert t1==1, "Error when input and output match"

# Should be 2/3 if input and output have one mismatch and no padding
t2 = masked_accuracy(y_true_test, y_pred_test)
print("t2=", t2)
assert t2==(2/3), "Error when input and output off-by-one, no padding"

# Add padding to y_true_test and y_pred_test
y_true_test_pad=np.array([[[1,0,0],[0,1,0],[1,0,0], [0,0,0],[0,0,0],[0,0,0],[0,0,0]]])
y_pred_test_pad=np.array([[[1,0,0],[0,1,0],[0,0,1], [0,0,0],[0,0,0],[0,0,0],[0,0,0]]])
print("y_pred_test_pad:\n",y_pred_test_pad)

# Should be 2/3 if input and output have one mismatch even with padding
t3 = masked_accuracy(y_true_test_pad, y_pred_test_pad)
print("t3=", t3)
assert t3==(2/3), "Error when input and output off-by-one, with padding"


t1= tf.Tensor(1.0, shape=(), dtype=float32)
t2= tf.Tensor(0.6666667, shape=(), dtype=float32)
y_pred_test_pad:
 [[[1 0 0]
  [0 1 0]
  [0 0 1]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
t3= tf.Tensor(0.6666667, shape=(), dtype=float32)


In [22]:
# model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=[masked_accuracy])

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    # shuffle = True,
)
# Save model
model.save("s2s_model.keras")

Epoch 1/5
101/101 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - loss: 1.6931 - masked_accuracy: 0.6802 - val_loss: 1.1391 - val_masked_accuracy: 0.7243
Epoch 2/5
101/101 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - loss: 1.0626 - masked_accuracy: 0.7272 - val_loss: 1.0535 - val_masked_accuracy: 0.7271
Epoch 3/5
101/101 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.9857 - masked_accuracy: 0.7340 - val_loss: 0.9226 - val_masked_accuracy: 0.7446
Epoch 4/5
101/101 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.9200 - masked_accuracy: 0.7469 - val_loss: 0.8514 - val_masked_accuracy: 0.7533
Epoch 5/5
101/101 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - loss: 0.8274 - masked_accuracy: 0.7660 - val_loss: 0.7696 - val_masked_accuracy: 0.7785


## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states

In [23]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [24]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence




In [25]:
# You can now generate decoded sentences as such:

for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("\n------")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence.strip())
    print("Reference sentence:", target_texts[seq_index].strip())




------
Input sentence: Those are good.
Decoded sentence: Cl e  ai  ai
Reference sentence: Ceux-ci sont bons.

------
Input sentence: I feel fine.
Decoded sentence: Je se  ai  ai  ai
Reference sentence: Je me sens bien.

------
Input sentence: I'm unemployed.
Decoded sentence: Je se  ai  ai  ai
Reference sentence: Je suis au chômage.

------
Input sentence: Hurry up.
Decoded sentence: Nle  ai  ai
Reference sentence: Magnez-vous !

------
Input sentence: Give it a try.
Decoded sentence: Nle  ai  ai
Reference sentence: Essaie.

------
Input sentence: Do it again!
Decoded sentence: Cl e  ai  ai  ai
Reference sentence: Faites-le de nouveau !

------
Input sentence: He stood up.
Decoded sentence: Cl e  ai  ai
Reference sentence: Il se mit debout.

------
Input sentence: Tom was silent.
Decoded sentence: Cl e  ai  ai  ai
Reference sentence: Tom gardait le silence.

------
Input sentence: I'm finished.
Decoded sentence: Je se  ai  ai  ai
Reference sentence: J'ai terminé.

------
Input sentenc